## 1. Datasets

In [14]:
import pandas as pd
import numpy


df_meta = pd.read_csv('./The_Movies_Dataset/movies_metadata.csv', low_memory=False)
df_ratings = pd.read_csv('./The_Movies_Dataset/ratings_small.csv', low_memory=False)


In [15]:
df_meta.head(3)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.015539,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.7129,/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0


In [16]:
df_ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


## 2. Limpieza

In [17]:
import pandas as pd
import numpy as np
import ast # Necesario para evaluar las cadenas JSON en 'genres'

# Configuración de Pandas para mejor visualización
pd.set_option('display.max_columns', None)

## ----------- 1: LIMPIEZA DE movies_metadata -------------------------

# low_memory=False se usa para evitar advertencias de pandas con datasets grandes y heterogéneos
df_meta = pd.read_csv('./The_Movies_Dataset/movies_metadata.csv', low_memory=False)

# 1. Limpieza de IDs 
df_meta = df_meta[df_meta['id'].str.isnumeric()]

df_meta['id'] = df_meta['id'].astype(int)
print(f"Filas después de limpiar IDs: {len(df_meta):,}")

# 2. Valores Faltantes  en 'overview'

df_meta['overview'] = df_meta['overview'].fillna('')
print("Vacios en 'overview' rellenados")

# 3. Parsing de la columna 'genres' (JSON)

def extract_names(json_string):
    if pd.isna(json_string) or json_string == '':
        return []
    try:
        # ast.literal_eval es más seguro que eval()
        list_of_dicts = ast.literal_eval(json_string)
        return [d['name'] for d in list_of_dicts]
    except:
        # En caso de error de parsing (malformación), devuelve vacío
        return []

df_meta['genres'] = df_meta['genres'].apply(extract_names)
print("Columna 'genres' parseada a una lista de nombres")

# 4. Selección de Columnas para el Sistema de Recomendación
df_movies_cleaned = df_meta[['id', 'title', 'overview', 'genres', 'vote_average', 'release_date']].copy()
df_movies_cleaned.rename(columns={'id': 'movieId'}, inplace=True) # Renombrar para hacer match con ratings

print(f"movies_metadata limpio: {len(df_movies_cleaned):,} registros\n")




Filas después de limpiar IDs: 45,463
Vacios en 'overview' rellenados
Columna 'genres' parseada a una lista de nombres
movies_metadata limpio: 45,463 registros



In [18]:
##  ------------2: LIMPIEZA DE ratings ------------------------

df_ratings = pd.read_csv('./The_Movies_Dataset/ratings_small.csv')

df_ratings['userId'] = df_ratings['userId'].astype(int)
df_ratings['movieId'] = df_ratings['movieId'].astype(int)

print(f"NaNs en ratings:\n{df_ratings.isnull().sum()}")

print(f"\nRegistros {len(df_ratings):,}")
print(f"Usuarios únicos: {df_ratings['userId'].nunique():,} \nPelículas únicas en ratings: {df_ratings['movieId'].nunique():,}\n")


NaNs en ratings:
userId       0
movieId      0
rating       0
timestamp    0
dtype: int64

Registros 100,004
Usuarios únicos: 671 
Películas únicas en ratings: 9,066



In [19]:
## --------------3: Merge de datasets ---------------------

# Hacemos merge (solo películas con ratings) para garantizar IDs válidos
df_merged = pd.merge(df_ratings, df_movies_cleaned, on='movieId', how='inner')
df_merged=df_merged.drop(['timestamp','release_date'], axis=1)

print("--- Fusión de Datasets (solo películas con ratings) ---")
print(f"Registros después del merge (ratings válidos con metadatos): {len(df_merged):,}")
print(f"Columnas resultantes: {df_merged.columns.tolist()}")

df_merged.head()

df_merged.to_csv('./archivos/dataset_merged.csv', index=False)
# df_movies_cleaned.to_csv('cleaned_movie_content.csv', index=False)

--- Fusión de Datasets (solo películas con ratings) ---
Registros después del merge (ratings válidos con metadatos): 44,994
Columnas resultantes: ['userId', 'movieId', 'rating', 'title', 'overview', 'genres', 'vote_average']
